In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import jieba
import re

In [2]:
class MyDataset(Dataset):
    def __init__(self, max_len, debug=True):
        super().__init__()
        df = pd.read_csv('../../datasets/THUCNews/train.csv')
        df = df.dropna()
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        else:
            df = df.sample(50000).reset_index(drop=True)
        # 读取常用停用词
        stopwords = [line.strip() for line in open('../../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
        sentences = []
        for title in df['title']:
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            # jieba分词
            sentence_seged = jieba.cut(title.strip())
            outstr = ''
            for word in sentence_seged:
                if word != '\t' and word not in stopwords:
                    outstr += word
                    outstr += ' '
            if outstr != '':
                sentences.append(outstr)
        # 获取所有词(token), <pad>用来填充不满足max_len的句子
        token_list = ['<pad>'] + list(set(' '.join(sentences).split()))
        # token和index互转字典
        self.token2idx = {token: i for i, token in enumerate(token_list)}
        self.idx2token = {i: token for i, token in enumerate(token_list)}
        self.vocab_size = len(self.token2idx)

        self.inputs = []
        for sentence in sentences:
            tokens = sentence.split()
            input_ = [self.token2idx[token] for token in tokens]
            if len(input_) < max_len:
                self.inputs.append(input_ + [self.token2idx['<pad>']] * (max_len - len(input_)))
            else:
                self.inputs.append(input_[: max_len])

        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])


class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 输入通道为1，卷成16通道输出，卷积核大小为(3*embed_size)，3类似于n-gram，可以换
        self.conv = nn.Conv2d(1, 16, (3, embed_size))
        self.dropout = nn.Dropout(0.2)
        # 输出头
        self.fc = nn.Linear(16, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        x = x.unsqueeze(1)  # [batch_size * 1 * 句子长度 * embed_size]，加一个维度，用于卷积层的输入
        x = self.conv(x)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽) * 1(卷积后的高)]
        x = x.squeeze(3)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽)] 压缩大小为1的维度
        x = torch.relu(x)  # 激活函数，尺寸不变
        x = torch.max_pool1d(x, x.size(2))  # 在每个通道做最大池化，[batch_size * 16(卷积层输出通道数) * 1]
        x = x.squeeze(2)  # 压缩维度2，[batch_size * 16(卷积层输出通道数)]
        x = self.dropout(x)  # dropout，尺寸不变
        logits = self.fc(x)  # 全连接输出头，[batch_size * n_class]
        return logits

In [3]:
dataset = MyDataset(max_len=10)  # 构造长度为10的句子输入，超过10的句子切掉，不足10的补<pad>
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
model = TextCNN(vocab_size=dataset.vocab_size, embed_size=128, n_class=dataset.n_class)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(100):
    for feature, target in dataloader:
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/d1/4_gsqv2176z583_7rmpm27lh0000gn/T/jieba.cache
Loading model cost 0.403 seconds.
Prefix dict has been built successfully.


epoch: 1 , loss: 2.773195743560791
epoch: 2 , loss: 1.8674651384353638
epoch: 3 , loss: 1.483340859413147
epoch: 4 , loss: 1.7299542427062988
epoch: 5 , loss: 1.6403207778930664
epoch: 6 , loss: 0.9790477752685547
epoch: 7 , loss: 1.2651853561401367
epoch: 8 , loss: 0.9305796027183533
epoch: 9 , loss: 0.6754975318908691
epoch: 10 , loss: 0.657089114189148
epoch: 11 , loss: 0.7448533773422241
epoch: 12 , loss: 0.6199911832809448
epoch: 13 , loss: 0.7406424880027771
epoch: 14 , loss: 0.3471713960170746
epoch: 15 , loss: 0.3989872932434082
epoch: 16 , loss: 0.2998690605163574
epoch: 17 , loss: 0.21135176718235016
epoch: 18 , loss: 0.6030945181846619
epoch: 19 , loss: 0.10659075528383255
epoch: 20 , loss: 0.23190981149673462
epoch: 21 , loss: 0.05693645030260086
epoch: 22 , loss: 0.14458638429641724
epoch: 23 , loss: 0.2682003676891327
epoch: 24 , loss: 0.41655316948890686
epoch: 25 , loss: 0.1977047473192215
epoch: 26 , loss: 0.06505148857831955
epoch: 27 , loss: 0.012891586869955063
epoc

In [4]:
df = pd.read_csv('../../datasets/THUCNews/train.csv')
predict = model(feature).max(1)[1].tolist()
for i in range(len(feature.tolist())):
    print(' '.join([dataset.idx2token[idx] for idx in feature.tolist()[i]]),
          '---> label:',
          dict(zip(df['label'], df['class']))[predict[i]])

奥巴马 获赠 成人 游戏 此类 已成 赢利 重点 <pad> <pad> ---> label: 游戏
三 小时 苦练 阿联 意犹未尽 队友 教练 离开 加练 <pad> ---> label: 体育
连跌 七天 年 最后 一个 交易日 期盼 惊喜 <pad> <pad> ---> label: 股票
业巡 精英 比洞 赛次 轮 彭婕 爆冷 出局 黄 永乐 ---> label: 体育
欧文 现身 曼联 基地 接受 体检 加盟 将成 定局 <pad> ---> label: 体育
顶级 变焦 牛头 佳能 赠镜 <pad> <pad> <pad> <pad> <pad> ---> label: 科技
广东 年 高考 录取 启动 提前 批 实行 平行 志愿 ---> label: 教育
新 托福 五大 特点 题型 应对 方法 <pad> <pad> <pad> ---> label: 教育
快讯 西飞 国际 尾盘 继续 诡异 狂 拉升 <pad> <pad> ---> label: 股票
韩国 决定 年 批量生产 电动汽车 <pad> <pad> <pad> <pad> <pad> ---> label: 时政
康健 平保 旗下 授 认股权 <pad> <pad> <pad> <pad> <pad> ---> label: 股票
安理会 决定 召开 紧急会议 讨论 朝鲜 核试验 <pad> <pad> <pad> ---> label: 时政
股指 回补 跳空 缺口 关注 后续 量 <pad> <pad> <pad> ---> label: 股票
环保部 今年 未 批复 环评 项目 涉及 亿 <pad> <pad> ---> label: 时政
百强 家具 新品 奢华 实 木家具 <pad> <pad> <pad> <pad> ---> label: 家居
探索 七界 万王 全球 热恋 醉汉 进化 <pad> <pad> <pad> ---> label: 游戏
